# StyleTTS2 モデル学習

各データセット(phone_min4, feat_top10, full100)に対して、ゼロからStyleTTS2モデルを学習します。

## 実行前の確認

1. **Cell 1-4を実行**: 環境とファイルの確認
   - PyTorch/transformersのバージョン確認
   - 必要なファイル（Utils/, Data/, Configs/）の存在確認
   - OOD_texts.txtの確認

2. **Colab環境の場合**: 
   - StyleTTS2リポジトリがクローンされていること
   - データセットと設定ファイルが準備されていること
   - `01_prepare_datasets.ipynb`と`02_generate_configs.ipynb`を実行済みであること

## 学習プロセス

1. **Stage 1 (train_first.py)**: 事前学習ステージ
   - エポック数: 80
   - Mixed precision (fp16) を使用
   - accelerate launch で実行

2. **Stage 2 (train_second.py)**: ジョイント学習ステージ
   - エポック数: 40
   - Stage 1のモデルをロードして継続学習

## 実行方法

**方法1: ノートブックから実行（推奨）**
- Cell 14で`RUN_STAGE1 = True`に設定して実行
- Stage 1完了後、`RUN_STAGE2 = True`に設定して実行

**方法2: ターミナルから実行**
- Cell 10で表示されるコマンドをターミナルで実行

## 注意事項

- 学習には長時間かかります（数時間〜数日）
- Colabの場合は、セッションがタイムアウトしないよう注意してください
- ブラウザを閉じると学習が中断されます
- TensorBoardで学習の進捗を確認できます
- VRAMエラーが発生した場合は、設定ファイルのbatch_sizeやmax_lenを削減してください


In [1]:
# PyTorchとtransformersのバージョン確認とアップグレード
# !pip install --upgrade torch>=2.6.0 torchaudio transformers>=4.40.0


In [2]:
import torch
import transformers
import subprocess

torch_version = torch.__version__
transformers_version = transformers.__version__

print(f"PyTorch version: {torch_version}")
print(f"transformers version: {transformers_version}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")

# バージョンチェック
torch_major, torch_minor = map(int, torch_version.split(".")[:2])
is_torch_ok = torch_major > 2 or (torch_major == 2 and torch_minor >= 6)

if not is_torch_ok:
    print(f"⚠️ 警告: PyTorch {torch_version} は2.6未満です。アップグレードが必要です。")
else:
    print(f"✓ PyTorch {torch_version} は2.6以上です。")

# accelerate設定の確認
print("\n" + "-"*60)
print("accelerate設定の確認")
print("-"*60)
try:
    result = subprocess.run(["accelerate", "config", "--config_file", "default"], 
                          capture_output=True, text=True, timeout=5)
    if "Inferred" in result.stdout or "default" in result.stdout.lower():
        print("✓ accelerate設定が存在します")
    else:
        print("⚠️ accelerate設定を確認できませんでした")
        print("必要に応じて以下を実行してください:")
        print("  accelerate config")
except:
    print("⚠️ accelerate configの確認に失敗しました")
    print("必要に応じて以下を実行してください:")
    print("  accelerate config")


PyTorch version: 2.9.1+cu128
transformers version: 4.57.3
CUDA available: True
GPU: NVIDIA GeForce RTX 4070 Ti
CUDA version: 12.8
✓ PyTorch 2.9.1+cu128 は2.6以上です。

------------------------------------------------------------
accelerate設定の確認
------------------------------------------------------------
⚠️ accelerate configの確認に失敗しました
必要に応じて以下を実行してください:
  accelerate config


In [3]:
import sys
import subprocess
from pathlib import Path
import os

# プロジェクトルートを取得
# notebooks/styletts2/ から実行される場合: 2階層上がる
# notebooks/ から実行される場合: 1階層上がる
current_dir = Path.cwd()
if current_dir.name == "styletts2":
    PROJECT_ROOT = current_dir.parent.parent  # notebooks/styletts2 -> notebooks -> project_root
elif current_dir.name == "notebooks":
    PROJECT_ROOT = current_dir.parent  # notebooks -> project_root
else:
    PROJECT_ROOT = current_dir  # プロジェクトルートから直接実行
STYLETTS2_DIR = PROJECT_ROOT / "StyleTTS2"

# StyleTTS2をパスに追加
if STYLETTS2_DIR.exists() and str(STYLETTS2_DIR) not in sys.path:
    sys.path.insert(0, str(STYLETTS2_DIR))

# データセット名のリスト
DATASETS = ["phone_min4", "feat_top10", "full100"]

print("="*60)
print("StyleTTS2 モデル学習")
print("="*60)
print(f"\nプロジェクトルート: {PROJECT_ROOT}")
print(f"StyleTTS2ディレクトリ: {STYLETTS2_DIR}")
print(f"データセット: {DATASETS}")
print(f"\n⚠️ 注意: 学習には長時間かかる可能性があります")
print(f"TensorBoardで進捗を確認できます: tensorboard --logdir {STYLETTS2_DIR}/Models")


StyleTTS2 モデル学習

プロジェクトルート: /mnt/e/dev/minimal-feature-corpus-tts
StyleTTS2ディレクトリ: /mnt/e/dev/minimal-feature-corpus-tts/StyleTTS2
データセット: ['phone_min4', 'feat_top10', 'full100']

⚠️ 注意: 学習には長時間かかる可能性があります
TensorBoardで進捗を確認できます: tensorboard --logdir /mnt/e/dev/minimal-feature-corpus-tts/StyleTTS2/Models


In [4]:
# 必要なファイルの存在確認
print("\n" + "="*60)
print("必要なファイルの存在確認")
print("="*60)

def check_file(path, description):
    """ファイルの存在を確認"""
    exists = path.exists()
    status = "✓" if exists else "✗"
    print(f"{status} {description}: {path}")
    if exists:
        if path.is_file():
            size = path.stat().st_size
            print(f"    サイズ: {size:,} bytes")
        elif path.is_dir():
            files = list(path.glob("*"))
            print(f"    ファイル数: {len(files)}")
    return exists

# StyleTTS2の必要なファイルを確認
required_files = {
    "F0モデル": STYLETTS2_DIR / "Utils" / "JDC" / "bst.t7",
    "ASR設定": STYLETTS2_DIR / "Utils" / "ASR" / "config.yml",
    "ASRモデル": STYLETTS2_DIR / "Utils" / "ASR" / "epoch_00080.pth",
    "PLBERTディレクトリ": STYLETTS2_DIR / "Utils" / "PLBERT",
    "OODテキスト": STYLETTS2_DIR / "Data" / "OOD_texts.txt",
}

all_files_exist = True
for desc, path in required_files.items():
    if not check_file(path, desc):
        all_files_exist = False

# データセットの確認
print("\n" + "-"*60)
print("データセットの確認")
print("-"*60)

for dataset_name in DATASETS:
    data_dir = STYLETTS2_DIR / "Data" / f"jvs002_{dataset_name}"
    train_list = data_dir / "train_list.txt"
    val_list = data_dir / "val_list.txt"
    wavs_dir = data_dir / "wavs"
    
    print(f"\n{dataset_name}:")
    check_file(data_dir, "データディレクトリ")
    check_file(train_list, "訓練データリスト")
    check_file(val_list, "検証データリスト")
    check_file(wavs_dir, "音声ファイルディレクトリ")
    
    if wavs_dir.exists():
        wav_files = list(wavs_dir.glob("*.wav"))
        print(f"    音声ファイル数: {len(wav_files)}")

# 設定ファイルの確認
print("\n" + "-"*60)
print("設定ファイルの確認")
print("-"*60)

for dataset_name in DATASETS:
    config_path = STYLETTS2_DIR / "Configs" / f"config_jvs002_{dataset_name}.yml"
    check_file(config_path, f"{dataset_name}設定ファイル")

# OOD_texts.txtの内容確認
print("\n" + "-"*60)
print("OOD_texts.txtの確認")
print("-"*60)
ood_path = STYLETTS2_DIR / "Data" / "OOD_texts.txt"
if ood_path.exists():
    with open(ood_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    print(f"✓ OOD_texts.txt: {len(lines)}行")
    if len(lines) > 0:
        print(f"  最初の3行の例:")
        for i, line in enumerate(lines[:3], 1):
            print(f"    {i}: {line.strip()[:80]}")
    # 日本語データセット用に、日本語テキストが含まれているか確認
    has_japanese = any(any('\u3040' <= c <= '\u309F' or '\u30A0' <= c <= '\u30FF' or '\u4E00' <= c <= '\u9FAF' for c in line) for line in lines[:100])
    if not has_japanese and len(lines) > 0:
        print("  ⚠️ 警告: OOD_texts.txtに日本語テキストが含まれていない可能性があります")
        print("    日本語データセットの場合、日本語テキストを含めることを推奨します")
else:
    print("✗ OOD_texts.txtが見つかりません")
    print("  形式: text|anything (1行1テキスト)")
    print("  例: これはテストテキストです|dummy")

# Colab環境でのファイル準備確認
print("\n" + "-"*60)
print("Colab環境でのファイル準備")
print("-"*60)
is_colab = str(STYLETTS2_DIR).startswith("/content")
if is_colab:
    print("⚠️ Colab環境が検出されました")
    print("\nColabでファイルを準備する方法:")
    print("  1. Google Driveにプロジェクトをマウント")
    print("  2. または、GitHubからクローン:")
    print(f"     !git clone https://github.com/yl4579/StyleTTS2.git {STYLETTS2_DIR}")
    print("  3. データセットと設定ファイルをアップロード")
    print("\n必要なファイル:")
    print("  - StyleTTS2/Utils/ (ASR, JDC, PLBERT)")
    print("  - StyleTTS2/Data/ (データセットとOOD_texts.txt)")
    print("  - StyleTTS2/Configs/ (設定ファイル)")

# まとめ
print("\n" + "="*60)
if all_files_exist:
    print("✓ すべての必要なファイルが存在します")
    print("\n次のステップ:")
    print("  1. Cell 14で学習を実行するか、ターミナルからコマンドを実行")
    print("  2. 学習の進捗をTensorBoardで確認")
else:
    print("✗ 一部のファイルが不足しています")
    print("\n不足しているファイルを準備してください:")
    missing_items = []
    if not (STYLETTS2_DIR / "Utils" / "JDC" / "bst.t7").exists():
        missing_items.append("Utils/JDC/bst.t7 (F0モデル)")
    if not (STYLETTS2_DIR / "Utils" / "ASR" / "config.yml").exists():
        missing_items.append("Utils/ASR/ (ASRモデル)")
    if not (STYLETTS2_DIR / "Utils" / "PLBERT").exists():
        missing_items.append("Utils/PLBERT/ (PLBERT)")
    if not (STYLETTS2_DIR / "Data" / "OOD_texts.txt").exists():
        missing_items.append("Data/OOD_texts.txt")
    
    for item in missing_items:
        print(f"  - {item}")
    
    if is_colab:
        print("\nColabでの準備方法:")
        print("  1. StyleTTS2リポジトリをクローン（Utils/が含まれます）")
        print("  2. データセットと設定ファイルを01_prepare_datasets.ipynbと02_generate_configs.ipynbで準備")
        print("  3. OOD_texts.txtを準備（日本語テキスト推奨）")
print("="*60)



必要なファイルの存在確認
✓ F0モデル: /mnt/e/dev/minimal-feature-corpus-tts/StyleTTS2/Utils/JDC/bst.t7
    サイズ: 21,029,926 bytes
✓ ASR設定: /mnt/e/dev/minimal-feature-corpus-tts/StyleTTS2/Utils/ASR/config.yml
    サイズ: 481 bytes
✓ ASRモデル: /mnt/e/dev/minimal-feature-corpus-tts/StyleTTS2/Utils/ASR/epoch_00080.pth
    サイズ: 94,552,811 bytes
✓ PLBERTディレクトリ: /mnt/e/dev/minimal-feature-corpus-tts/StyleTTS2/Utils/PLBERT
    ファイル数: 4
✓ OODテキスト: /mnt/e/dev/minimal-feature-corpus-tts/StyleTTS2/Data/OOD_texts.txt
    サイズ: 31,758,898 bytes

------------------------------------------------------------
データセットの確認
------------------------------------------------------------

phone_min4:
✓ データディレクトリ: /mnt/e/dev/minimal-feature-corpus-tts/StyleTTS2/Data/jvs002_phone_min4
    ファイル数: 3
✓ 訓練データリスト: /mnt/e/dev/minimal-feature-corpus-tts/StyleTTS2/Data/jvs002_phone_min4/train_list.txt
    サイズ: 1,693 bytes
✓ 検証データリスト: /mnt/e/dev/minimal-feature-corpus-tts/StyleTTS2/Data/jvs002_phone_min4/val_list.txt
    サイズ: 911 bytes
✓ 音声ファイル

In [5]:
import sys
import subprocess
from pathlib import Path
import os

# プロジェクトルートを取得
# プロジェクトルートを取得
# notebooks/styletts2/ から実行される場合: 2階層上がる
# notebooks/ から実行される場合: 1階層上がる
current_dir = Path.cwd()
if current_dir.name == "styletts2":
    PROJECT_ROOT = current_dir.parent.parent  # notebooks/styletts2 -> notebooks -> project_root
elif current_dir.name == "notebooks":
    PROJECT_ROOT = current_dir.parent  # notebooks -> project_root
else:
    PROJECT_ROOT = current_dir  # プロジェクトルートから直接実行
STYLETTS2_DIR = PROJECT_ROOT / "StyleTTS2"

# StyleTTS2をパスに追加
if STYLETTS2_DIR.exists() and str(STYLETTS2_DIR) not in sys.path:
    sys.path.insert(0, str(STYLETTS2_DIR))

# データセット名のリスト
DATASETS = ["phone_min4", "feat_top10", "full100"]

print("="*60)
print("StyleTTS2 モデル学習")
print("="*60)
print(f"\nプロジェクトルート: {PROJECT_ROOT}")
print(f"StyleTTS2ディレクトリ: {STYLETTS2_DIR}")
print(f"データセット: {DATASETS}")
print(f"\n⚠️ 注意: 学習には長時間かかる可能性があります")
print(f"TensorBoardで進捗を確認できます: tensorboard --logdir {STYLETTS2_DIR}/Models")


StyleTTS2 モデル学習

プロジェクトルート: /mnt/e/dev/minimal-feature-corpus-tts
StyleTTS2ディレクトリ: /mnt/e/dev/minimal-feature-corpus-tts/StyleTTS2
データセット: ['phone_min4', 'feat_top10', 'full100']

⚠️ 注意: 学習には長時間かかる可能性があります
TensorBoardで進捗を確認できます: tensorboard --logdir /mnt/e/dev/minimal-feature-corpus-tts/StyleTTS2/Models


In [6]:
# 設定ファイルとデータセットの確認
print("\n" + "-"*60)
print("設定ファイルとデータセットの確認")
print("-"*60)

for dataset_name in DATASETS:
    config_path = STYLETTS2_DIR / "Configs" / f"config_jvs002_{dataset_name}.yml"
    data_dir = STYLETTS2_DIR / "Data" / f"jvs002_{dataset_name}"
    
    print(f"\n{dataset_name}:")
    print(f"  設定ファイル: {config_path} ({'存在' if config_path.exists() else '不存在'})")
    print(f"  データディレクトリ: {data_dir} ({'存在' if data_dir.exists() else '不存在'})")
    
    if data_dir.exists():
        train_list = data_dir / "train_list.txt"
        val_list = data_dir / "val_list.txt"
        wavs_dir = data_dir / "wavs"
        
        train_count = len(open(train_list).readlines()) if train_list.exists() else 0
        val_count = len(open(val_list).readlines()) if val_list.exists() else 0
        wav_count = len(list(wavs_dir.glob("*.wav"))) if wavs_dir.exists() else 0
        
        print(f"    訓練データ: {train_count}エントリ")
        print(f"    検証データ: {val_count}エントリ")
        print(f"    音声ファイル: {wav_count}ファイル")



------------------------------------------------------------
設定ファイルとデータセットの確認
------------------------------------------------------------

phone_min4:
  設定ファイル: /mnt/e/dev/minimal-feature-corpus-tts/StyleTTS2/Configs/config_jvs002_phone_min4.yml (存在)
  データディレクトリ: /mnt/e/dev/minimal-feature-corpus-tts/StyleTTS2/Data/jvs002_phone_min4 (存在)
    訓練データ: 3エントリ
    検証データ: 1エントリ
    音声ファイル: 4ファイル

feat_top10:
  設定ファイル: /mnt/e/dev/minimal-feature-corpus-tts/StyleTTS2/Configs/config_jvs002_feat_top10.yml (存在)
  データディレクトリ: /mnt/e/dev/minimal-feature-corpus-tts/StyleTTS2/Data/jvs002_feat_top10 (存在)
    訓練データ: 9エントリ
    検証データ: 1エントリ
    音声ファイル: 10ファイル

full100:
  設定ファイル: /mnt/e/dev/minimal-feature-corpus-tts/StyleTTS2/Configs/config_jvs002_full100.yml (存在)
  データディレクトリ: /mnt/e/dev/minimal-feature-corpus-tts/StyleTTS2/Data/jvs002_full100 (存在)
    訓練データ: 90エントリ
    検証データ: 10エントリ
    音声ファイル: 100ファイル


In [7]:
# Stage 1学習コマンド生成関数
def generate_stage1_command(dataset_name: str) -> str:
    """
    Stage 1学習のコマンドを生成
    
    Args:
        dataset_name: データセット名
        
    Returns:
        学習コマンド文字列
    """
    config_path = STYLETTS2_DIR / "Configs" / f"config_jvs002_{dataset_name}.yml"
    
    # 相対パスに変換（StyleTTS2ディレクトリから見たパス）
    config_rel_path = config_path.relative_to(STYLETTS2_DIR)
    config_rel_path_str = str(config_rel_path).replace('\\', '/')  # Windows対応
    
    cmd = f"accelerate launch --mixed_precision=fp16 train_first.py -p {config_rel_path_str}"
    
    return cmd

print("✓ Stage 1学習コマンド生成関数を定義しました")


✓ Stage 1学習コマンド生成関数を定義しました


In [8]:
# Stage 2学習コマンド生成関数
def generate_stage2_command(dataset_name: str) -> str:
    """
    Stage 2学習のコマンドを生成
    
    Args:
        dataset_name: データセット名
        
    Returns:
        学習コマンド文字列
    """
    config_path = STYLETTS2_DIR / "Configs" / f"config_jvs002_{dataset_name}.yml"
    
    # 相対パスに変換（StyleTTS2ディレクトリから見たパス）
    config_rel_path = config_path.relative_to(STYLETTS2_DIR)
    config_rel_path_str = str(config_rel_path).replace('\\', '/')  # Windows対応
    
    cmd = f"python train_second.py -p {config_rel_path_str}"
    
    return cmd

print("✓ Stage 2学習コマンド生成関数を定義しました")


✓ Stage 2学習コマンド生成関数を定義しました


## 学習コマンドの表示

以下のコマンドをStyleTTS2ディレクトリで実行してください。各データセットごとに実行します。


In [9]:
# 学習コマンドの表示
print("="*60)
print("学習コマンド（参考）")
print("="*60)
print("\n以下のコマンドをStyleTTS2ディレクトリで実行してください。")
print("各データセットごとに実行します。")
print("\n⚠️ 注意:")
print("  - 学習には長時間かかります（数時間〜数日）")
print("  - Colabの場合は、セッションがタイムアウトしないよう注意してください")
print("  - 進捗はTensorBoardで確認できます")

for dataset_name in DATASETS:
    print(f"\n{'='*60}")
    print(f"データセット: {dataset_name}")
    print(f"{'='*60}")
    
    print(f"\n【Stage 1学習】")
    print(f"cd {STYLETTS2_DIR}")
    stage1_cmd = generate_stage1_command(dataset_name)
    print(f"{stage1_cmd}")
    print(f"\n  または、このノートブックのCell 14で実行:")
    print(f"    dataset_name = \"{dataset_name}\"")
    print(f"    RUN_STAGE1 = True")
    print(f"    run_stage1_training(dataset_name, execute=True)")
    
    print(f"\n【Stage 2学習】 (Stage 1完了後)")
    print(f"cd {STYLETTS2_DIR}")
    stage2_cmd = generate_stage2_command(dataset_name)
    print(f"{stage2_cmd}")
    print(f"\n  または、このノートブックのCell 14で実行:")
    print(f"    dataset_name = \"{dataset_name}\"")
    print(f"    RUN_STAGE2 = True")
    print(f"    run_stage2_training(dataset_name, execute=True)")
    
    print(f"\n学習ログとチェックポイントは以下に保存されます:")
    print(f"  {STYLETTS2_DIR / 'Models' / f'jvs002_{dataset_name}'}")
    print(f"  - ログ: {STYLETTS2_DIR / 'Models' / f'jvs002_{dataset_name}' / 'train.log'}")
    print(f"  - TensorBoard: tensorboard --logdir {STYLETTS2_DIR / 'Models' / f'jvs002_{dataset_name}'}")


学習コマンド（参考）

以下のコマンドをStyleTTS2ディレクトリで実行してください。
各データセットごとに実行します。

⚠️ 注意:
  - 学習には長時間かかります（数時間〜数日）
  - Colabの場合は、セッションがタイムアウトしないよう注意してください
  - 進捗はTensorBoardで確認できます

データセット: phone_min4

【Stage 1学習】
cd /mnt/e/dev/minimal-feature-corpus-tts/StyleTTS2
accelerate launch --mixed_precision=fp16 train_first.py -p Configs/config_jvs002_phone_min4.yml

  または、このノートブックのCell 14で実行:
    dataset_name = "phone_min4"
    RUN_STAGE1 = True
    run_stage1_training(dataset_name, execute=True)

【Stage 2学習】 (Stage 1完了後)
cd /mnt/e/dev/minimal-feature-corpus-tts/StyleTTS2
python train_second.py -p Configs/config_jvs002_phone_min4.yml

  または、このノートブックのCell 14で実行:
    dataset_name = "phone_min4"
    RUN_STAGE2 = True
    run_stage2_training(dataset_name, execute=True)

学習ログとチェックポイントは以下に保存されます:
  /mnt/e/dev/minimal-feature-corpus-tts/StyleTTS2/Models/jvs002_phone_min4
  - ログ: /mnt/e/dev/minimal-feature-corpus-tts/StyleTTS2/Models/jvs002_phone_min4/train.log
  - TensorBoard: tensorboard --logdir /mnt/e/dev/minimal-f

## プログラム実行による学習（オプション）

以下のセルでプログラムから直接学習を実行することもできます。ただし、Jupyter Notebookで長時間実行する場合、ブラウザを閉じると中断される可能性があるため、コマンドライン実行を推奨します。


In [10]:
# Stage 1学習実行関数
def run_stage1_training(dataset_name: str, execute: bool = False):
    """
    Stage 1学習を実行
    
    Args:
        dataset_name: データセット名
        execute: Trueの場合、実際に実行する。Falseの場合、コマンドのみ表示
    """
    config_path = STYLETTS2_DIR / "Configs" / f"config_jvs002_{dataset_name}.yml"
    
    if not config_path.exists():
        print(f"✗ 設定ファイルが見つかりません: {config_path}")
        return
    
    # 相対パスに変換（StyleTTS2ディレクトリから見たパス）
    config_rel_path = config_path.relative_to(STYLETTS2_DIR)
    config_rel_path_str = str(config_rel_path).replace('\\', '/')
    
    # #region agent log
    import json
    import os
    import sys
    from datetime import datetime
    log_path = "/mnt/e/dev/minimal-feature-corpus-tts/.cursor/debug.log"
    def log_debug(location, message, data, hypothesis_id):
        try:
            with open(log_path, "a", encoding="utf-8") as f:
                f.write(json.dumps({
                    "sessionId": "debug-session",
                    "runId": "run1",
                    "hypothesisId": hypothesis_id,
                    "location": location,
                    "message": message,
                    "data": data,
                    "timestamp": int(datetime.now().timestamp() * 1000)
                }, ensure_ascii=False) + "\n")
        except: pass
    # #endregion
    
    # #region agent log
    log_debug("03_train_models.ipynb:814", "Checking accelerate command availability", {"cmd": "accelerate"}, "A")
    # #endregion
    
    # accelerateコマンドの検出を試行
    accelerate_cmd = None
    accelerate_paths = []
    
    # #region agent log
    log_debug("03_train_models.ipynb:820", "Testing accelerate detection methods", {"method": "which"}, "A")
    # #endregion
    
    # 方法1: which accelerate
    try:
        result = subprocess.run(["which", "accelerate"], capture_output=True, text=True, timeout=2)
        if result.returncode == 0 and result.stdout.strip():
            accelerate_cmd = result.stdout.strip()
            accelerate_paths.append(("which", accelerate_cmd))
            # #region agent log
            log_debug("03_train_models.ipynb:828", "accelerate found via which", {"path": accelerate_cmd}, "A")
            # #endregion
    except Exception as e:
        # #region agent log
        log_debug("03_train_models.ipynb:832", "which accelerate failed", {"error": str(e)}, "A")
        # #endregion
        pass
    
    # #region agent log
    log_debug("03_train_models.ipynb:836", "Testing python -m accelerate", {"method": "python -m"}, "B")
    # #endregion
    
    # 方法2: python -m accelerate
    if not accelerate_cmd:
        try:
            result = subprocess.run(["python", "-m", "accelerate", "--version"], capture_output=True, text=True, timeout=2)
            if result.returncode == 0:
                accelerate_cmd = "python"
                accelerate_paths.append(("python -m", "python -m accelerate"))
                # #region agent log
                log_debug("03_train_models.ipynb:844", "accelerate found via python -m", {"method": "python -m"}, "B")
                # #endregion
        except Exception as e:
            # #region agent log
            log_debug("03_train_models.ipynb:848", "python -m accelerate failed", {"error": str(e)}, "B")
            # #endregion
            pass
    
    # #region agent log
    log_debug("03_train_models.ipynb:852", "Testing python3 -m accelerate", {"method": "python3 -m"}, "C")
    # #endregion
    
    # 方法3: python3 -m accelerate
    if not accelerate_cmd:
        try:
            result = subprocess.run(["python3", "-m", "accelerate", "--version"], capture_output=True, text=True, timeout=2)
            if result.returncode == 0:
                accelerate_cmd = "python3"
                accelerate_paths.append(("python3 -m", "python3 -m accelerate"))
                # #region agent log
                log_debug("03_train_models.ipynb:860", "accelerate found via python3 -m", {"method": "python3 -m"}, "C")
                # #endregion
        except Exception as e:
            # #region agent log
            log_debug("03_train_models.ipynb:864", "python3 -m accelerate failed", {"error": str(e)}, "C")
            # #endregion
            pass
    
    # #region agent log
    log_debug("03_train_models.ipynb:868", "Testing pip list for accelerate", {"method": "pip list"}, "D")
    # #endregion
    
    # 方法4: pip listで確認
    try:
        result = subprocess.run(["pip", "list"], capture_output=True, text=True, timeout=5)
        has_accelerate = "accelerate" in result.stdout.lower()
        # #region agent log
        log_debug("03_train_models.ipynb:875", "pip list result", {"has_accelerate": has_accelerate, "stdout_preview": result.stdout[:200]}, "D")
        # #endregion
    except Exception as e:
        # #region agent log
        log_debug("03_train_models.ipynb:879", "pip list failed", {"error": str(e)}, "D")
        # #endregion
        has_accelerate = False
    
    # #region agent log
    log_debug("03_train_models.ipynb:883", "Checking sys.executable", {"executable": sys.executable}, "E")
    # #endregion
    
    # 方法5: sys.executableのbinディレクトリからaccelerateを探す
    if not accelerate_cmd:
        try:
            import pathlib
            bin_dir = pathlib.Path(sys.executable).parent
            accelerate_path = bin_dir / "accelerate"
            if accelerate_path.exists() and accelerate_path.is_file():
                accelerate_cmd = str(accelerate_path)
                accelerate_paths.append(("sys.executable bin", accelerate_cmd))
                # #region agent log
                log_debug("03_train_models.ipynb:891", "accelerate found in bin directory", {"path": accelerate_cmd}, "E")
                # #endregion
        except Exception as e:
            # #region agent log
            log_debug("03_train_models.ipynb:895", "bin directory search failed", {"error": str(e), "executable": sys.executable}, "E")
            # #endregion
            pass
    
    # 方法6: sys.executable -m accelerate（フォールバック）
    if not accelerate_cmd:
        try:
            # accelerateコマンドの存在確認（--versionは使えないので、代わりにhelpを使用）
            result = subprocess.run([sys.executable, "-m", "accelerate", "launch", "--help"], capture_output=True, text=True, timeout=2)
            if result.returncode == 0 or "accelerate" in result.stderr.lower() or "accelerate" in result.stdout.lower():
                accelerate_cmd = sys.executable
                accelerate_paths.append(("sys.executable -m", f"{sys.executable} -m accelerate"))
                # #region agent log
                log_debug("03_train_models.ipynb:905", "accelerate found via sys.executable -m", {"executable": sys.executable}, "E")
                # #endregion
        except Exception as e:
            # #region agent log
            log_debug("03_train_models.ipynb:909", "sys.executable -m accelerate failed", {"error": str(e), "executable": sys.executable}, "E")
            # #endregion
            pass
    
    # #region agent log
    log_debug("03_train_models.ipynb:899", "Final accelerate command decision", {"accelerate_cmd": accelerate_cmd, "paths_tried": accelerate_paths}, "ALL")
    # #endregion
    
    # コマンドを構築
    if accelerate_cmd == "python" or accelerate_cmd == "python3" or (accelerate_cmd and accelerate_cmd.endswith("python")):
        # python -m accelerate を使用
        cmd = [accelerate_cmd, "-m", "accelerate", "launch", "--mixed_precision=fp16", "train_first.py", "-p", config_rel_path_str]
    elif accelerate_cmd and (accelerate_cmd.endswith("accelerate") or "/accelerate" in accelerate_cmd):
        # 直接パスを使用（binディレクトリから見つかった場合）
        cmd = [accelerate_cmd, "launch", "--mixed_precision=fp16", "train_first.py", "-p", config_rel_path_str]
    elif accelerate_cmd:
        # sys.executable -m accelerate を使用
        cmd = [accelerate_cmd, "-m", "accelerate", "launch", "--mixed_precision=fp16", "train_first.py", "-p", config_rel_path_str]
    else:
        # デフォルト（エラーになる可能性がある）
        cmd = ["accelerate", "launch", "--mixed_precision=fp16", "train_first.py", "-p", config_rel_path_str]
    
    print(f"\n{'='*60}")
    print(f"Stage 1学習開始: {dataset_name}")
    print(f"{'='*60}")
    print(f"コマンド: {' '.join(cmd)}")
    print(f"作業ディレクトリ: {STYLETTS2_DIR}")
    if accelerate_cmd:
        print(f"✓ accelerateコマンド検出: {accelerate_cmd}")
    else:
        print(f"⚠️ accelerateコマンドが見つかりませんでした")
        print(f"  試行した方法: {[p[0] for p in accelerate_paths]}")
    print(f"\n⚠️ 注意: 学習には長時間かかります（数時間〜数日）")
    print(f"進捗は以下で確認できます:")
    print(f"  - ログ: {STYLETTS2_DIR / 'Models' / f'jvs002_{dataset_name}' / 'train.log'}")
    print(f"  - TensorBoard: tensorboard --logdir {STYLETTS2_DIR / 'Models' / f'jvs002_{dataset_name}'}")
    
    if execute:
        print(f"\n学習を開始します...")
        print(f"⚠️ 長時間実行されるため、ブラウザを閉じないでください")
        print(f"⚠️ Colabの場合は、セッションがタイムアウトしないよう注意してください\n")
        
        # #region agent log
        log_debug("03_train_models.ipynb:925", "About to execute subprocess.Popen", {"cmd": cmd, "cwd": str(STYLETTS2_DIR)}, "ALL")
        # #endregion
        
        try:
            # リアルタイム出力のために、stdout/stderrを直接表示
            process = subprocess.Popen(
                cmd,
                cwd=str(STYLETTS2_DIR),
                stdout=subprocess.PIPE,
                stderr=subprocess.STDOUT,
                text=True,
                bufsize=1,
                universal_newlines=True
            )
            
            # #region agent log
            log_debug("03_train_models.ipynb:938", "subprocess.Popen created", {"pid": process.pid}, "ALL")
            # #endregion
            
            # リアルタイムで出力を表示
            if process.stdout:
                for line in process.stdout:
                    print(line, end='')
            
            process.wait()
            
            if process.returncode == 0:
                print(f"\n{'='*60}")
                print(f"✓ Stage 1学習完了: {dataset_name}")
                print(f"{'='*60}")
            else:
                print(f"\n{'='*60}")
                print(f"✗ Stage 1学習に失敗: {dataset_name} (終了コード: {process.returncode})")
                print(f"{'='*60}")
                print(f"\nエラーログを確認してください:")
                log_path = STYLETTS2_DIR / "Models" / f"jvs002_{dataset_name}" / "train.log"
                if log_path.exists():
                    print(f"  {log_path}")
        except KeyboardInterrupt:
            print(f"\n⚠️ 学習が中断されました")
            # #region agent log
            log_debug("03_train_models.ipynb:950", "KeyboardInterrupt caught", {}, "ALL")
            # #endregion
        except Exception as e:
            print(f"\n✗ エラー: {e}")
            # #region agent log
            log_debug("03_train_models.ipynb:954", "Exception caught in subprocess.Popen", {"error": str(e), "error_type": type(e).__name__}, "ALL")
            # #endregion
            import traceback
            traceback.print_exc()
    else:
        print("\n⚠️ execute=Falseのため、実際には実行しません。")
        print("実行する場合は、execute=Trueを指定してください。")
        print("\nまたは、以下のコマンドをターミナルで実行してください:")
        print(f"  cd {STYLETTS2_DIR}")
        print(f"  {' '.join(cmd)}")

print("✓ Stage 1学習実行関数を定義しました")


✓ Stage 1学習実行関数を定義しました


In [11]:
# Stage 2学習実行関数
def run_stage2_training(dataset_name: str, execute: bool = False):
    """
    Stage 2学習を実行
    
    Args:
        dataset_name: データセット名
        execute: Trueの場合、実際に実行する。Falseの場合、コマンドのみ表示
    """
    config_path = STYLETTS2_DIR / "Configs" / f"config_jvs002_{dataset_name}.yml"
    
    if not config_path.exists():
        print(f"✗ 設定ファイルが見つかりません: {config_path}")
        return
    
    # Stage 1のチェックポイントを確認
    model_dir = STYLETTS2_DIR / "Models" / f"jvs002_{dataset_name}"
    stage1_checkpoints = list(model_dir.glob("epoch_1st_*.pth")) if model_dir.exists() else []
    if not stage1_checkpoints and execute:
        print(f"⚠️ 警告: Stage 1のチェックポイントが見つかりません")
        print(f"  Stage 1を先に実行してください")
        print(f"  確認先: {model_dir}")
        return
    
    # 相対パスに変換（StyleTTS2ディレクトリから見たパス）
    config_rel_path = config_path.relative_to(STYLETTS2_DIR)
    config_rel_path_str = str(config_rel_path).replace('\\', '/')
    
    cmd = ["python", "train_second.py", "-p", config_rel_path_str]
    
    print(f"\n{'='*60}")
    print(f"Stage 2学習開始: {dataset_name}")
    print(f"{'='*60}")
    print(f"コマンド: {' '.join(cmd)}")
    print(f"作業ディレクトリ: {STYLETTS2_DIR}")
    if stage1_checkpoints:
        latest_checkpoint = sorted(stage1_checkpoints)[-1]
        print(f"最新のStage 1チェックポイント: {latest_checkpoint.name}")
    print(f"\n⚠️ 注意: 学習には長時間かかります（数時間〜数日）")
    print(f"進捗は以下で確認できます:")
    print(f"  - ログ: {STYLETTS2_DIR / 'Models' / f'jvs002_{dataset_name}' / 'train.log'}")
    print(f"  - TensorBoard: tensorboard --logdir {STYLETTS2_DIR / 'Models' / f'jvs002_{dataset_name}'}")
    
    if execute:
        print(f"\n学習を開始します...")
        print(f"⚠️ 長時間実行されるため、ブラウザを閉じないでください")
        print(f"⚠️ Colabの場合は、セッションがタイムアウトしないよう注意してください\n")
        
        try:
            # リアルタイム出力のために、stdout/stderrを直接表示
            process = subprocess.Popen(
                cmd,
                cwd=str(STYLETTS2_DIR),
                stdout=subprocess.PIPE,
                stderr=subprocess.STDOUT,
                text=True,
                bufsize=1,
                universal_newlines=True
            )
            
            # リアルタイムで出力を表示
            if process.stdout:
                for line in process.stdout:
                    print(line, end='')
            
            process.wait()
            
            if process.returncode == 0:
                print(f"\n{'='*60}")
                print(f"✓ Stage 2学習完了: {dataset_name}")
                print(f"{'='*60}")
            else:
                print(f"\n{'='*60}")
                print(f"✗ Stage 2学習に失敗: {dataset_name} (終了コード: {process.returncode})")
                print(f"{'='*60}")
                print(f"\nエラーログを確認してください:")
                log_path = STYLETTS2_DIR / "Models" / f"jvs002_{dataset_name}" / "train.log"
                if log_path.exists():
                    print(f"  {log_path}")
        except KeyboardInterrupt:
            print(f"\n⚠️ 学習が中断されました")
        except Exception as e:
            print(f"\n✗ エラー: {e}")
            import traceback
            traceback.print_exc()
    else:
        print("\n⚠️ execute=Falseのため、実際には実行しません。")
        print("実行する場合は、execute=Trueを指定してください。")
        print("\nまたは、以下のコマンドをターミナルで実行してください:")
        print(f"  cd {STYLETTS2_DIR}")
        print(f"  {' '.join(cmd)}")

print("✓ Stage 2学習実行関数を定義しました")


✓ Stage 2学習実行関数を定義しました


In [31]:
# 学習実行
# ⚠️ 注意: 実行すると長時間かかります（数時間〜数日）。各データセットごとに個別に実行してください。

print("="*60)
print("学習実行")
print("="*60)
print("\n実行するデータセットを選択してください:")
print("  - phone_min4: 最小データセット（4文、検証用）")
print("  - feat_top10: 中規模データセット（10文）")
print("  - full100: 全データセット（100文）")
print("\n⚠️ 注意事項:")
print("  - 学習には長時間かかります（数時間〜数日）")
print("  - Colabの場合は、セッションがタイムアウトしないよう注意してください")
print("  - ブラウザを閉じると学習が中断されます")
print("  - 進捗はTensorBoardで確認できます")

# 実行するデータセットを指定
dataset_name = "phone_min4"  # 変更可能: "feat_top10", "full100"
# Stage 1 Done: "phone_min4", "feat_top10"
# Stage 2 Done: 

# 実行フラグ（Trueにすると実際に実行されます）
RUN_STAGE1 = False  # Stage 1学習を実行する場合は True に変更
RUN_STAGE2 = True  # Stage 2学習を実行する場合は True に変更（Stage 1完了後）

print(f"\n{'='*60}")
print(f"実行設定: {dataset_name}")
print(f"{'='*60}")
print(f"Stage 1実行: {RUN_STAGE1}")
print(f"Stage 2実行: {RUN_STAGE2}")

if RUN_STAGE1:
    print(f"\n{'='*60}")
    print(f"Stage 1学習を開始します: {dataset_name}")
    print(f"{'='*60}")
    run_stage1_training(dataset_name, execute=True)
elif RUN_STAGE2:
    print(f"\n{'='*60}")
    print(f"Stage 2学習を開始します: {dataset_name}")
    print(f"{'='*60}")
    print("⚠️ 注意: Stage 1が完了していることを確認してください")
    run_stage2_training(dataset_name, execute=True)
else:
    print("\n⚠️ 実行フラグがFalseのため、学習は実行されません。")
    print("\n学習を実行するには:")
    print("  1. RUN_STAGE1 = True に設定してStage 1を実行")
    print("  2. Stage 1完了後、RUN_STAGE2 = True に設定してStage 2を実行")
    print("\nまたは、以下のコマンドをターミナルで実行:")
    print(f"  cd {STYLETTS2_DIR}")
    print(f"  accelerate launch --mixed_precision=fp16 train_first.py -p Configs/config_jvs002_{dataset_name}.yml")


学習実行

実行するデータセットを選択してください:
  - phone_min4: 最小データセット（4文、検証用）
  - feat_top10: 中規模データセット（10文）
  - full100: 全データセット（100文）

⚠️ 注意事項:
  - 学習には長時間かかります（数時間〜数日）
  - Colabの場合は、セッションがタイムアウトしないよう注意してください
  - ブラウザを閉じると学習が中断されます
  - 進捗はTensorBoardで確認できます

実行設定: phone_min4
Stage 1実行: False
Stage 2実行: True

Stage 2学習を開始します: phone_min4
⚠️ 注意: Stage 1が完了していることを確認してください

Stage 2学習開始: phone_min4
コマンド: python train_second.py -p Configs/config_jvs002_phone_min4.yml
作業ディレクトリ: /mnt/e/dev/minimal-feature-corpus-tts/StyleTTS2
最新のStage 1チェックポイント: epoch_1st_00070.pth

⚠️ 注意: 学習には長時間かかります（数時間〜数日）
進捗は以下で確認できます:
  - ログ: /mnt/e/dev/minimal-feature-corpus-tts/StyleTTS2/Models/jvs002_phone_min4/train.log
  - TensorBoard: tensorboard --logdir /mnt/e/dev/minimal-feature-corpus-tts/StyleTTS2/Models/jvs002_phone_min4

学習を開始します...
⚠️ 長時間実行されるため、ブラウザを閉じないでください
⚠️ Colabの場合は、セッションがタイムアウトしないよう注意してください

Loading the first stage model at Models/jvs002_phone_min4/first_stage.pth ...
decoder loaded
text_encoder loaded
style_encoder

In [32]:
# 学習済みモデルの確認
print("\n" + "-"*60)
print("学習済みモデルの確認")
print("-"*60)

for dataset_name in DATASETS:
    model_dir = STYLETTS2_DIR / "Models" / f"jvs002_{dataset_name}"
    
    if model_dir.exists():
        print(f"\n{dataset_name}:")
        print(f"  モデルディレクトリ: {model_dir}")
        
        # チェックポイントファイルを検索
        checkpoint_files = list(model_dir.glob("*.pth"))
        if checkpoint_files:
            print(f"  チェックポイント: {len(checkpoint_files)}ファイル")
            for cp in sorted(checkpoint_files)[:5]:  # 最初の5つを表示
                print(f"    - {cp.name}")
            if len(checkpoint_files) > 5:
                print(f"    ... 他 {len(checkpoint_files) - 5}ファイル")
        else:
            print(f"  チェックポイント: なし（学習未完了）")
        
        # TensorBoardログの確認
        tb_dir = model_dir / "tensorboard"
        if tb_dir.exists():
            print(f"  TensorBoardログ: {tb_dir}")
    else:
        print(f"\n{dataset_name}: モデルディレクトリが存在しません（学習未開始）")



------------------------------------------------------------
学習済みモデルの確認
------------------------------------------------------------

phone_min4:
  モデルディレクトリ: /mnt/e/dev/minimal-feature-corpus-tts/StyleTTS2/Models/jvs002_phone_min4
  チェックポイント: 13ファイル
    - epoch_1st_00000.pth
    - epoch_1st_00010.pth
    - epoch_1st_00020.pth
    - epoch_1st_00030.pth
    - epoch_1st_00040.pth
    ... 他 8ファイル
  TensorBoardログ: /mnt/e/dev/minimal-feature-corpus-tts/StyleTTS2/Models/jvs002_phone_min4/tensorboard

feat_top10:
  モデルディレクトリ: /mnt/e/dev/minimal-feature-corpus-tts/StyleTTS2/Models/jvs002_feat_top10
  チェックポイント: 9ファイル
    - epoch_1st_00000.pth
    - epoch_1st_00010.pth
    - epoch_1st_00020.pth
    - epoch_1st_00030.pth
    - epoch_1st_00040.pth
    ... 他 4ファイル
  TensorBoardログ: /mnt/e/dev/minimal-feature-corpus-tts/StyleTTS2/Models/jvs002_feat_top10/tensorboard

full100:
  モデルディレクトリ: /mnt/e/dev/minimal-feature-corpus-tts/StyleTTS2/Models/jvs002_full100
  チェックポイント: 4ファイル
    - epoch_1st_00000.pt

## 学習完了後の確認

学習が完了したら、以下のノートブックで評価音声を生成できます:
- `04_generate_evaluation_audio.ipynb`


## 学習時間の比較

複数のデータセットの学習時間を比較できます。


In [33]:
# 学習時間の比較
import sys
sys.path.insert(0, str(STYLETTS2_DIR))

from compare_training_times import compare_training_times

# 学習時間を比較
compare_training_times(str(STYLETTS2_DIR / "Models"))


学習時間比較

データセット               総時間             平均エポック時間             エポック数      開始時刻                
----------------------------------------------------------------------------------------------------
jvs002_phone_min4    7分36秒           3秒                   80         2026-01-09 01:07:39 
jvs002_feat_top10    5時間24分14秒       3分58秒                80         2026-01-08 17:44:35 
jvs002_full100       N/A             N/A                  31         2026-01-08 07:23:56 

詳細情報


jvs002_phone_min4:
  総学習時間: 0.13時間 (7.6分)
  平均エポック時間: 0.00時間 (0.1分)
  エポック数: 80
  開始時刻: 2026-01-09 01:07:39
  終了時刻: 2026-01-09 01:15:15
  エポック時間サンプル:
    Epoch 1: 0.00時間 (0.1分)
    Epoch 2: 0.00時間 (0.1分)
    Epoch 3: 0.00時間 (0.1分)
    ...
    Epoch 78: 0.00時間 (0.1分)
    Epoch 79: 0.00時間 (0.1分)
    Epoch 80: 0.00時間 (0.1分)

jvs002_feat_top10:
  総学習時間: 5.40時間 (324.2分)
  平均エポック時間: 0.07時間 (4.0分)
  エポック数: 80
  開始時刻: 2026-01-08 17:44:35
  終了時刻: 2026-01-08 23:08:50
  エポック時間サンプル:
    Epoch 1: 0.00時間 (0.1分)
    Epoch 2: 0.00時間 

TypeError: unsupported format string passed to NoneType.__format__